# Dolby.io Developer Days Media APIs 101 - Getting Started
### Notebook #2: Getting Started

**Now that we have made our first API call, lets take a step back...**

## **What is an API?**

- An Application Programming Interface or API can be thought of as a tool in a builder's toolbox. 
- When using an API, we format and send data over a server, which is then interpreted and returned to the user. 
- By outsourcing various tasks or "tools" to a 3rd-party, users or builders can focus on the big picture.

## **<div> What About REST APIs?</div>**

- Much like tools in a toolbox, there are different types of APIs. 
- In the case of Dolby.io our APIs conform to the constraints of representational state transfer architecture or REST for short. 
- There is a lot to say about REST APIs, however, we'll keep it simple with a few key takeaways.

![rest](./images/rest.png)

**Structure**

In [ ]:
# URL used as the endpoint
url = "https://api.dolby.com/media/enhance"

# HTTP headers passed on, content type, api key, etc.
headers = {
  "x-api-key": "api_key",
  "Content-Type": "application/json",
  "Accept": "application/json"
}

# Body content of larger data
body = {
  "input" : "dlb://in/example.mp4",
  "output" : "dlb://out/example-enhanced.mp4",
}

# URL parameters added on
params = {
  "job_id": job_id
}

# HTTP method, POST, GET, PUT, DELETE, etc.
response = requests.post(url, json=body, headers=headers, params=params)

For further reading, check out this article: https://restfulapi.net/

## What can we use the Dolby.io Media APIs for?

First, let's set our variables again.

In [ ]:
import IPython

# Enter your Dolby.io Media API Key here.
api_key = "<YOUR_API_KEY_HERE>"
# Enter your name here to customize the output URL later.
name = "<YOUR_NAME_HERE>"

## Upload local files

Let's say our files are not currently accessible on a public URL. While you can use your cloud storage providers to [presign URLs temporarily](https://docs.dolby.io/media-apis/docs/media-input-and-output), this doesn't solve the local file issue. In comes the Input APIs.

In [ ]:
# Example Local File
IPython.display.Audio("white_noise_demo.wav")

In [ ]:
import requests

# Set or replace these values

file_path = "white_noise_demo.wav"
input_url = f"dlb://in/input-{name}.wav" # Setting the input URL to have a different location based on your name!

url = "https://api.dolby.com/media/input"
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

# Declare the dlb:// location we want to store the file in
body = {
    "url": input_url,
}

response = requests.post(url, json=body, headers=headers)
response.raise_for_status()
data = response.json()
presigned_url = data["url"]

# Upload your media to the pre-signed url response

print("Uploading {0} to {1}".format(file_path, presigned_url))
with open(file_path, "rb") as input_file:
  requests.put(presigned_url, data=input_file)

And now the file can be accessed via our new `dlb://` url. This media is stored in an isolated location that only you can access with your API key. It is encrypted at rest and while in transit with https. Access to the production system is controlled and audited to prevent unauthorized use.
> Note: this storage is TEMPORARY and will be removed within 24 - 48 hours.

### Enhance

In part one, we called enhance without using any parameters. We can fine tune Enhance with a few different parameters to optimize for common audio issues including:

- Noise reduction
- Speech leveling and isolation
- Loudness correction
- Content tuning
- Sibilance reduction
- Plosive reduction
- Dynamic equalization
- Tone shaping
- Hum reduction
- Mouth click reduction

**When defining the body**
- Input location
- Output location

Input and output can be a cloud storage provider.

- Content type
    - conference
    - interview
    - lecture
    - meeting
    - mobile_phone
    - music
    - podcast
    - studio
    - voice_over or voice_recording
    
Content type can be left undefined. Enhance will work it out.

- Audio parameters
    - loudness
    - dynamics
    - noise
    - filter
    - speech
    - music
    
Audio parameters can be left undefined, all objects will default to enabled.

Audio parameters have sub-features, for example:
- Speech
    - isolation 
    - sibilance
    - plosive
    - click

**Lots to choose from depending on your use case**

**Let's see an example**

In [ ]:
#URL changes
url = "https://api.dolby.com/media/enhance"

output_url = f"dlb://out/output-{name}.wav" # Setting the input URL to have a different location based on your name!

#Header stays the same
headers = {
  "x-api-key": api_key,
  "Content-Type": "application/json",
  "Accept": "application/json"
}

body = {
  "input" : input_url,
  "output" : output_url,
  "content" : {
      "type": "voice_over"},
  "audio" : {
      "noise": {"reduction": {
          "amount": "max"
      }
    },
     "filter":{"hum": {
         "enable": True
     }
    },
      "speech": {
            "isolation": {
                "enable": True,
                "amount": 70
            },
            "sibilance": {"reduction": {
                    "enable": True,
                    "amount": "medium"
                }},
            "plosive": {"reduction": {
                    "enable": True,
                    "amount": "medium"
                }},
            "click": {"reduction": {
                    "enable": False,
                    "amount": "medium"
    }}}
  }
}

response = requests.post(url, json=body, headers=headers)
print(response.json())

**Checking the status once again:**

In [ ]:
params = {
  "job_id": response.json()["job_id"]
}

stat = requests.get(url, params=params, headers=headers)
print(stat.json())

**Download the output**

In [ ]:
url = "https://api.dolby.com/media/output"

params = {
    "url": output_url
}

import shutil #Package for file operations
output_path = "white_noise_demo-enhanced.wav"

with requests.get(url, params=params, headers=headers, stream=True) as response:
    response.raw.decode_content = True
    print("Downloading from {0} into {1}".format(response.url, output_path))
    with open(output_path, "wb") as output_file:
        shutil.copyfileobj(response.raw, output_file)

**Lets give it a listen**

In [ ]:
IPython.display.Audio("white_noise_demo-enhanced.wav")

**Original**

In [ ]:
IPython.display.Audio("white_noise_demo.wav")

### Questions?

https://docs.dolby.io/media-apis/docs/enhance-api-guide

### Diagnose (*Beta*)

- Provides General Media Info
- Audio Quality 
- Noise Score
- Clipping
- Loudness
- Content Classification

Diagnose is **fast**, and doesn't require a file to be downloaded - the results are in the same API call as the status.

**Let's look at some code**

In [ ]:
# Note the new URL endpoint
url = "https://api.dolby.com/media/diagnose"

headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

body = {
    "input"  : input_url
}

response = requests.post(url, json=body, headers=headers)
job_id  = response.json()["job_id"]
print(response.json())

**Checking Status and Getting Results**

In [ ]:
url = "https://api.dolby.com/media/diagnose"
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

params = {
    "job_id" : job_id,
}

stat = requests.get(url, params=params, headers=headers)
print(stat.json())

In [ ]:
stat.json()

![flow](./images/flow.jpg)

### A quick note on cURL & our API reference

- Designed to transfer data using various network protocols
- Works great with REST APIs like the Dolby.io Media APIs
- Run through the command line

**Lets check out the Docs**
- https://docs.dolby.io/media-apis/reference/media-diagnose-post
- https://docs.dolby.io/media-apis/reference/media-diagnose-get

Easiest to do in Postman Collection:
https://www.postman.com/dolbyio/workspace/dolby-io-media-apis/overview

In [ ]:
#Shell POST Example
curl --request POST \
     --url https://api.dolby.com/media/diagnose \
     --header 'Accept: application/json' \
     --header 'Content-Type: application/json' \
     --header 'x-api-key: YOUR_API_KEY' \
     --data '
{
     "input": "string",
     "content": {
          "type": "podcast",
          "silence": {
               "threshold": -60,
               "duration": 3
          }
     }
}
'

In [ ]:
#Powershell POST Example
$headers=@{}
$headers.Add("Accept", "application/json")
$headers.Add("Content-Type", "application/json")
$headers.Add("x-api-key", "YOUR_API_KEY")
$response = Invoke-WebRequest -Uri 'https://api.dolby.com/media/diagnose' -Method POST -Headers $headers -ContentType 'application/json' -Body '{"content":{"silence":{"threshold":-60,"duration":3},"type":"podcast"},"input":"dlb://in/example.wav"}'

#Then
$response

### Questions?

https://docs.dolby.io/media-apis/docs/diagnose-api-guide

## What about other languages?

**JavaScript**

**Keep your API keys private**

https://docs.dolby.io/media-apis/docs/authentication

**Kotlin**

Check out our API Reference:
https://docs.dolby.io/media-apis/reference/media-enhance-post

### Transcoding (*Beta*)

- Convert media of a particular file type to another
- Transizing video media to different resolutions
- Transrating audio media to different sample rates or bit rates

The goal of Transcoding is to convert from one media format to another, from one media format to many variations, or in generating streaming media such as HLS or DASH.

![formats](./images/formats.png)

- Containers host video and audio compression codecs.
- HLS: HTTP Live Streaming
- DASH: Dynamic Adaptive Streaming over HTTP

**Lets check out some code**

In [ ]:
url = "https://api.dolby.com/media/transcode"

headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json",
}


In [ ]:
body = {
    "inputs": [{ "source" : "https://dolbyio.s3-us-west-1.amazonaws.com/public/shelby/indoors.original.mp4"}],
    "outputs": [
        {
            "id" : "my_mp4",
            "destination" : "dlb://out/airplane-transcoded.wav",
            "kind" : "wav"
        }
    ]
}

In [ ]:
response = requests.post(url, json=body, headers=headers)
print(response.json())

In [ ]:
params = {
    "job_id": response.json()["job_id"]
}
stat = requests.get(url, params=params, headers=headers)
print(stat.json())

In [ ]:
url = "https://api.dolby.com/media/output"
output_path = "airplane-example.wav"
params = {
    "url": "dlb://out/airplane-transcoded.wav",
}

with requests.get(url, params=params, headers=headers, stream=True) as response:
    response.raise_for_status()
    response.raw.decode_content = True
    print("Downloading from {0} into {1}".format(response.url, output_path))
    with open(output_path, "wb") as output_file:
        shutil.copyfileobj(response.raw, output_file)

In [ ]:
!ls

### Questions?

https://docs.dolby.io/media-apis/docs/transcode-api-guide

### More APIs?

**Analyze (*Beta*) and Analyze Speech (*Beta*)**

- Bigger version of Diagnose

- Breaks out many segments of the audio for quality scores

- Speaker diarization

- Micro level audio defects and insights

**We'll explore Analyze and Analyze Speech in another workshop so stay tuned and let us know if you are interested.**

Explore more here: https://docs.dolby.io/media-apis/docs/analyze-api-guide

**Music Mastering (*available at request, soon-to-be GA*)**

- Like Enhance but more music specific

- Pick between various music styles

- Professional-sounding audio masters that keep creative intent intact

**If you are interested in Music Mastering reach out to the team!**

Explore more here: https://dolby.io/products/music-mastering/

**Webhooks**

**To Recap...**

- Enhance API
    - Noise reduction
    - Speaker isolation
    - Explored how regions work

- Diagnose API
    - Quality scores
    - Learning more about your media
    - Played with cURL and learnt about some other languages

- Transcoding API
    - Media streams
    - Converting to different codecs

- Breifly touched on Analyze and Music Mastering APIs

**Thank you for attending the Media API segment of Dolby.io Developer Days!**

If you have questions feel free to reach out to me at:
*@BradenRiggs1* or */in/briggs599*

Complete this survey to enter our raffle for some Dolby.io t-shirts: https://dolbyioprograms.typeform.com/to/S4H65blD

**Your Feedback is Appreciated**